## PLX - Phuture Memecoin Index - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import Methodology
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  0.25e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "meme-token"

# Calculating max slippage based on liquidity threshold
liq = 2.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = Methodology(min_mcap,
                                min_weight,
                                max_weight,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    )
liquidity

Excluding corgiai, pricing data available only for 149 < 180 days
Note: harrypotterobamasonic10in, marketcap data available only for 134 < 180 days
Excluding wall-street-memes, pricing data available only for 70 < 180 days
0xe5b49820e5a1063f6f4ddf851327b5e8b2301048
Dropping harrypotterobamasonic10in because a price feed is unavailable
Dropping jesus-coin because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
pepe,6.328514e+05,6.420195e+05,0.014487,ethereum
dogelon-mars,5.192898e+06,5.158242e+06,-0.006674,ethereum
baby-doge-coin,7.560985e+08,7.435360e+08,-0.016615,binance-smart-chain
shiba-inu,9.936645e+04,9.751604e+04,-0.018622,ethereum
harrypotterobamasonic10in,8.129454e+00,7.932555e+00,-0.024221,ethereum
floki,2.513797e+04,2.452066e+04,-0.024557,ethereum
jesus-coin,3.177598e+06,2.957089e+06,-0.069395,ethereum
leash,3.043056e-03,2.757859e-03,-0.093721,ethereum
dejitaru-tsuka,3.377559e+01,2.990739e+01,-0.114526,ethereum


In [8]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
shiba-inu,Shiba Inu,5.880436e+09,0.300000,77,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
pepe,Pepe,6.501137e+08,0.300000,77,0x6982508145454ce325ddbe47a25d4ec3d2311933,ethereum
floki,FLOKI,3.897286e+08,0.233926,59,0xcf0c122c6b73ff809c693db761e7baebe62b6a2e,ethereum
baby-doge-coin,Baby Doge Coin,2.085637e+08,0.110373,28,0xc748673057861a797275cd8a068abb95a902e8de,binance-smart-chain
dogelon-mars,Dogelon Mars,1.065524e+08,0.055701,14,0x761d38e5ddf6ccf6cf7c55759d5210750b5d60f3,ethereum
